In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
path = "drive/MyDrive/645-project-data/"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# !pip install autogluon
# !pip install --upgrade mxnet

In [ ]:
import pandas as pd
import os
import shutil
import autogluon as ag
from autogluon.vision import ImagePredictor, ImageDataset

In [ ]:
labels = pd.read_csv(path + "labels.csv")
labels = labels.rename(columns={"id":"image", "breed":"label"})
labels['image'] = labels['image'].apply(lambda x: os.path.join(path + "train", x))
labels['image'] = labels['image'].apply(lambda x: x + '.jpg')

data = ImageDataset(labels)

In [ ]:
predictor = ImagePredictor()
predictor.fit(data, time_limit=60*120, hyperparameters={'epochs': 100}) 

ImagePredictor sets accuracy as default eval_metric for classification problems.
Reset labels to ['affenpinscher', 'afghan_hound', 'african_hunting_dog', 'airedale', 'american_staffordshire_terrier', 'appenzeller', 'australian_terrier', 'basenji', 'basset', 'beagle', 'bedlington_terrier', 'bernese_mountain_dog', 'black-and-tan_coonhound', 'blenheim_spaniel', 'bloodhound', 'bluetick', 'border_collie', 'border_terrier', 'borzoi', 'boston_bull', 'bouvier_des_flandres', 'boxer', 'brabancon_griffon', 'briard', 'brittany_spaniel', 'bull_mastiff', 'cairn', 'cardigan', 'chesapeake_bay_retriever', 'chihuahua', 'chow', 'clumber', 'cocker_spaniel', 'collie', 'curly-coated_retriever', 'dandie_dinmont', 'dhole', 'dingo', 'doberman', 'english_foxhound', 'english_setter', 'english_springer', 'entlebucher', 'eskimo_dog', 'flat-coated_retriever', 'french_bulldog', 'german_shepherd', 'german_short-haired_pointer', 'giant_schnauzer', 'golden_retriever', 'gordon_setter', 'great_dane', 'great_pyrenees', 'g

In [ ]:
fit_result = predictor.fit_summary()
print('Top-1 train acc: %.3f, val acc: %.3f' %(fit_result['train_acc'], fit_result['valid_acc']))

Top-1 train acc: 0.855, val acc: 0.899


In [ ]:
labels_test = pd.read_csv(path + "test-labels.csv")
labels_test = labels_test.rename(columns={"id":"image", "breed":"label"})
labels_test['image'] = labels_test['image'].apply(lambda x: os.path.join("drive/MyDrive/645-project-data/test_2", x))

test_dataset = ImageDataset(labels_test)

In [ ]:
image_path = test_dataset.iloc[11]['image']
result = predictor.predict(image_path)
print(result)

0    bouvier_des_flandres
Name: label, dtype: object


In [ ]:
bulk_result = predictor.predict(test_dataset)
results = labels_test.copy()
results["preds"] = bulk_result
test_accuracy = (((results["preds"] == results["label"]).sum())/results.shape[0]) * 100
print('test acc: %.3f' % test_accuracy + "%")

# test_acc = predictor.evaluate(test_dataset)
# print('Top-1 test acc: %.3f' % test_acc['top1'])

test acc: 92.308%


In [ ]:
filename = 'ag_predictor_86T90V.ag'
predictor.save(filename)
shutil.copy(f"/content/{filename}", path + "models/" + filename)